In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [45]:
for page in range(1,5):
    print(f'page {page}\n')
    url = f'https://www.yell.com/ucs/UcsSearchAction.do?find=Y&keywords=restaurants&location=boston&pageNum={page}'
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'}
    r = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    articles = soup.find_all('div', class_='businessCapsule--mainRow')

    for item in articles:
        name = item.find('span', class_='businessCapsule--name').text
        type_of_restaurant = item.find('span', class_='businessCapsule--classification').text
        more_info_link = item.find('a', class_='businessCapsule--moreInfoBtn')['href']
        more_info_link = f'https://www.yell.com{more_info_link}'
        r2 = requests.get(url=more_info_link, headers=headers)
        print('more info request sent')
        soup2 = BeautifulSoup(r2.content, 'html.parser')
        print('sleeping 5 sec')
        time.sleep(5)
        try:
            address = soup2.find('span', {'itemprop' : 'address'}).text.strip().replace('\n','')
        except:
            address = -1
        try:
            website = soup2.find('a', class_='businessCard--callToAction')['href']
        except:
            website = -1
        
        restaurants = {
            'name' : name,
            'type of restaurant' : type_of_restaurant,
            'address' : address,
            'website' : website
        }
        print('sleeping 5 sec')
        time.sleep(5)
    print('sleeping 10 sec \n')
    time.sleep(10)
df = pd.DataFrame(restaurants)
df.to_csv('restaurant_list.csv', index=False)
    